# PePR: Direct Generalized Membership Inference Attack
Attack a single target model with a singe attack configuration.

## Prep Google Colab Enviornment
**Important: Restart the Runtime after this Cell!**
The restart is needed because of `pip install -e`.

In [ ]:
!git clone https://github.com/hallojs/ml-pepr.git
%pip install -e ml-pepr
%pip install pylatex

Cloning into 'ml-pepr'...
remote: Enumerating objects: 279, done.
remote: Counting objects: 100% (279/279), done.
remote: Compressing objects: 100% (193/193), done.
remote: Total 279 (delta 142), reused 210 (delta 83), pack-reused 0
Receiving objects: 100% (279/279), 290.84 KiB | 24.24 MiB/s, done.
Resolving deltas: 100% (142/142), done.
Obtaining file:///content/ml-pepr
  Running setup.py develop for pepr
     |████████████████████████████████| 92kB 9.6MB/s 
  Created wheel for pylatex: filename=PyLaTeX-1.4.1-cp36-none-any.whl size=42814 sha256=1519222353161b30dfcba92b6c8d60717db2d668d0ac293d4e33b1457077d7cf
  Stored in directory: /root/.cache/pip/wheels/a2/25/3a/2cc0a6219d95ce34f1f7439a6427c62ca262ebaeb5969db89f
  Created wheel for ordered-set: filename=ordered_set-4.0.2-py2.py3-none-any.whl size=8210 sha256=895615baf9c55cf7087223d80df139c7440fd8363851ad370fa3660ce04e2c6e
  Stored in directory: /root/.cache/pip/wheels/e1/c6/9b/651d8a21d59b51a75ab9c070838f9231b8126421bc0569af47
Succes

## Imports

In [ ]:
from pepr.privacy import gmia
from pepr.utilities import assign_record_ids_to_target_models

import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense

import numpy as np
import logging

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## Setup Logging

In [ ]:
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s', '%Y-%m-%d %H:%M:%S')

# TensorFlow Logger
file_handler_tf = logging.FileHandler('tf.log')
file_handler_tf.setLevel(logging.INFO)
file_handler_tf.setFormatter(formatter)

tf.get_logger().setLevel(logging.INFO)
logger_tf = tf.get_logger()
logger_tf.addHandler(file_handler_tf)

# PePR Logger
file_handler_pr = logging.FileHandler('pepr.privacy.gmia.log')
file_handler_pr.setLevel(logging.DEBUG)
file_handler_pr.setFormatter(formatter)

stream_handler_pr = logging.StreamHandler()
stream_handler_pr.setLevel(logging.DEBUG)
stream_handler_pr.setFormatter(formatter)

logger_pr = logging.getLogger('pepr.privacy.gmia')
logger_pr.addHandler(file_handler_pr)
logger_pr.addHandler(stream_handler_pr)

## A Few Function Definitions

In [ ]:
def create_model(input_shape, n_categories):
  """Architecture of the target and reference models.

  Parameters
  ----------
  input_shape : tuple
      Dimensions of the input for the target/training
  n_categories : int
      number of categories for the prediction
  models.

  Returns
  -------
  tensorflow.python.keras.engine.sequential.Sequential
      A convolutional neuronal network model.
  """
  model = Sequential()

  # first convolution layer
  model.add(Conv2D(filters=32, kernel_size=(5, 5), strides=(
      1, 1), padding='same', input_shape=input_shape))
  model.add(Activation('relu'))

  # max pooling layer
  model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

  # second convolution layer
  model.add(Conv2D(filters=64, kernel_size=(
      5, 5), strides=(1, 1), padding='same'))
  model.add(Activation('relu'))

  # max pooling layer
  model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

  # fully connected layer
  model.add(Flatten())
  model.add(Dense(1024))
  model.add(Activation('relu'))

  # drop out
  model.add(Dropout(rate=0.5))

  # fully connected layer
  model.add(Dense(n_categories))
  model.add(Activation('softmax'))

  return model

def create_compile_model():
  """Create compiled model.
  
  At the moment pepr.gmia needs this function to train the reference models.

  Returns
  -------
    tensorflow.python.keras.engine.sequential.Sequential
      A compiled tensorflow model.
  """
  input_shape = (28, 28, 1)
  number_classes = 10

  model = create_model(input_shape, number_classes)

  optimizer = optimizers.Adam(lr=0.0001)
  loss = 'categorical_crossentropy'
  metrics = ["accuracy"]
  model.compile(optimizer, loss=loss, metrics=metrics)

  return model

def load_fashion_mnist():
    """Loads and preprocesses the fashion mnist dataset.

    Returns
    -------
    tuple
        (training data, training labels, test data, test labels)
    """
    train, test = tf.keras.datasets.fashion_mnist.load_data()
    train_data, train_labels = train
    test_data, test_labels = test

    # Normalize the data to a range between 0 and 1
    train_data = np.array(train_data, dtype=np.float32) / 255
    test_data = np.array(test_data, dtype=np.float32) / 255

    # Reshape the images to (28, 28, 1)
    train_data = train_data.reshape(train_data.shape[0], 28, 28, 1)
    test_data = test_data.reshape(test_data.shape[0], 28, 28, 1)

    train_labels = np.array(train_labels, dtype=np.int32)
    test_labels = np.array(test_labels, dtype=np.int32)

    return np.vstack((train_data, test_data)), np.hstack((train_labels, test_labels))

## Data Setup

In [ ]:
data, labels = load_fashion_mnist()

4423680/4422102 [==============================] - 0s 0us/step


## Train a Target Model

In [ ]:
target_model = create_compile_model()
target_model.fit(data[40000:50000],
                 tf.keras.utils.to_categorical(labels[40000:50000], num_classes=10),
                 epochs=50,
                 batch_size=50,
                 verbose=0)
target_model.save('data/target_model')

INFO:tensorflow:Assets written to: data/target_model/assets


## Run the Attack

In [ ]:
attack_pars = {
    'number_classes': 10,
    'number_reference_models': 100,
    'reference_training_set_size': 10000,
    'create_compile_model': create_compile_model,
    'reference_epochs': 50,
    'reference_batch_size': 50,
    'hlf_metric': 'cosine',
    'hlf_layer_number': 10,
    'number_target_records': 25
}

# single target
data_conf = {
    'reference_indices': list(range(40000)),
    'target_indices': list(range(40000, 50000)),
    'evaluation_indices': list(range(40000, 60000)),
    'record_indices_per_target': np.array([np.arange(10000)])
}

gmia_attack = gmia.DirectGmia('Playground GMIA', attack_pars, data, labels, data_conf, [target_model])

gmia_attack.run(save_path='data')

gmia_attack.create_attack_report()

2021-01-08 17:16:34 - pepr.privacy.gmia - INFO - Create mapping of records to reference models.
2021-01-08 17:16:34 - pepr.privacy.gmia - INFO - Save mapping of records to reference models: data/records_per_reference_model.npy.
2021-01-08 17:16:34 - pepr.privacy.gmia - DEBUG - records_per_reference_model shape: (100, 10000)
2021-01-08 17:16:34 - pepr.privacy.gmia - INFO - Progress: Train reference model 1/100.
2021-01-08 17:17:02 - pepr.privacy.gmia - INFO - Progress: Train reference model 2/100.
2021-01-08 17:17:31 - pepr.privacy.gmia - INFO - Progress: Train reference model 3/100.
2021-01-08 17:18:01 - pepr.privacy.gmia - INFO - Progress: Train reference model 4/100.
2021-01-08 17:18:29 - pepr.privacy.gmia - INFO - Progress: Train reference model 5/100.
2021-01-08 17:18:58 - pepr.privacy.gmia - INFO - Progress: Train reference model 6/100.
2021-01-08 17:19:28 - pepr.privacy.gmia - INFO - Progress: Train reference model 7/100.
2021-01-08 17:19:56 - pepr.privacy.gmia - INFO - Progress:

INFO:tensorflow:Assets written to: data/reference_model0/assets


2021-01-08 18:04:56 - pepr.privacy.gmia - INFO - Save reference model: data/reference_model1.


INFO:tensorflow:Assets written to: data/reference_model1/assets


2021-01-08 18:04:57 - pepr.privacy.gmia - INFO - Save reference model: data/reference_model2.


...

2021-01-08 18:13:23 - pepr.privacy.gmia - INFO - Do inference on model 98/100.


2021-01-08 18:13:23 - pepr.privacy.gmia - INFO - Do inference on model 99/100.


2021-01-08 18:13:23 - pepr.privacy.gmia - INFO - Do inference on model 100/100.


2021-01-08 18:13:23 - pepr.privacy.gmia - DEBUG - Reference inferences shape: (25, 100)
2021-01-08 18:13:23 - pepr.privacy.gmia - INFO - Infer log losses of target model.
2021-01-08 18:13:23 - pepr.privacy.gmia - INFO - Do inference on model 1/1.


2021-01-08 18:13:23 - pepr.privacy.gmia - DEBUG - Target inferences shape: (25, 1)
2021-01-08 18:13:23 - pepr.privacy.gmia - INFO - Sample reference losses, approximate empirical cumulative distribution function, smooth ecdf with piecewise cubic interpolation.
2021-01-08 18:13:23 - pepr.privacy.gmia - INFO - Number of used target records: 25.
2021-01-08 18:13:23 - pepr.privacy.gmia - INFO - Used target records (indexes): [ 1144  1930  2521  3304  4326  4380  4570  6064  8453  9709  9777 10846
 12961 13165 14078 14220 14405 15569 16435 16522 18097 18380 18420 19168
 19860].
2021-01-08 18:13:23 - pepr.privacy.gmia - DEBUG - PCHIP references shape: 25
2021-01-08 18:13:23 - pepr.privacy.gmia - DEBUG - ECDF references shape: 25
2021-01-08 18:13:23 - pepr.privacy.gmia - INFO - Determine members and non-members with left-tailed hypothesis test.
2021-01-08 18:13:23 - pepr.privacy.gmia - DEBUG - Members determined for the 0-th target model: []
2021-01-08 18:13:23 - pepr.privacy.gmia - DEBUG - M

In [ ]:
# Zip report directory if you want to download it from google colab
!zip -r gmia_report.zip gmia_report